In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

## Import Statements

In [2]:
import json
from sklearn.cluster import KMeans
from sklearn.model_selection import cross_val_score #how good model is 
from sklearn.model_selection import cross_val_predict
import sklearn.ensemble as skens
from sklearn.decomposition import PCA
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from IPython.display import Image
from sklearn.metrics import classification_report, confusion_matrix
#visualization imports 

## 1) Put data in the correct format in order to run RandomForestClassifier later in the code: 

- load in train.json data from kaggle 

In [ ]:
train = pd.read_json('/kaggle/input/whats-cooking/train.json')
train.head()

In [ ]:
test = pd.read_json('/kaggle/input/whats-cooking/test.json')
test.head()

#### Explanation: 
The dataset in both train and test is in a format that cannot be used in a Random Forest Classifier. For the train data, the columns need to be every unique ingredient with a 0 or 1 if the ingredient is in a particular cuisine. In the next part, re-load both the test data and train data, and create a dictionary that will be used to create a DataFrame with the information formatted correctly. Do the same thing for the test data as well. 

The test.json data does not have cuisines listed. We are going to use the random forest classifier that is going to be created below on the train.json data. Since the test.json does not have cusines listed, I am going to run cross validation on this data to get an accuracy score. 

## 2) Re-read in json files to be cleaned before putting it into pandas DataFrames. 
- read in test.json and assign it to the variable test_data 
- read in train.json and assign it to the variable data 

In [ ]:
with open('/kaggle/input/whats-cooking/train.json') as json_file:
    data = json.load(json_file)
    
with open('/kaggle/input/whats-cooking/test.json') as json_file:
    test_data = json.load(json_file)

#### a) Get a list of all the unique ingredients in both data (train data) and test_data (test data) and assign the resulting lists to two separate variables.  The train data ingredients, which comes from the values in the variable data, will be a assigned to a list called unique_ingredients. The test data ingredients, which comes from the values in the variable test_data, will be assigned to a list called unique_ingredients_test.

In [ ]:
#train data 
lst = []
for x in data: 
    id_=x["id"]
    ingredients = x['ingredients']
    for ingred in ingredients:
        lst.append(ingred)

In [ ]:
#train data ingredients 
unique_ingredients=list(set(lst))

In [ ]:
#test data
lst2 = []
for x in test_data: 
    id_=x["id"]
    ingredients = x['ingredients']
    for ingred in ingredients:
        lst2.append(ingred)


In [ ]:
#test data ingredients 
unique_ingredients_test =list(set(lst2))

#### b) Create a dictionary called ingred_value that formats the train data correctly, called data, to be used for a Random Forest Classifier. This dictionary will be used to create a DataFrame called train_df.

This is a dictionary called ingred_value where one of the keys is called 'cusine' and the value is a list of all the cuisines in the dataset. The other keys in the dictionary is all the ingredients in the dataset as keys. The values are 0 or 1 depending on if the particular cuisine mentioned has the ingredient. 

In [ ]:
ingred_value = {ingredient: [] for ingredient in unique_ingredients}

In [ ]:
ids = []
ciusine = [] 
ingredient_lst = unique_ingredients
ingredients_for_id =[] 
for x in data: 
    id_=x["id"]
    ingredients_in_id = x['ingredients']
    ids.append(id_)
    ciusine.append(x["cuisine"])
    for ingred in ingred_value: 
            if ingred in ingredients_in_id:
                ingred_value[ingred].append(1)
            else: 
                 ingred_value[ingred].append(0)

                

In [ ]:
# create a key in the ingred_value dictionary called cuisine 
# The value is now a list of cuisines in the dataset 
# the list called ciusine was created above 

ingred_value["cuisine"] = ciusine

#### c) Created a DataFrame called train_df with the ingred_value dictionary that was created above. 

In [ ]:
train_df = pd.DataFrame(ingred_value)

In [ ]:
train_df.head()

##### Explanation: 
The DataFrame train_df is now formatted correctly to perform machine learning classifiers on. 

#### d) Repeat steps b and c above for test_data. Create a dictionary called ingred_value_test that formats the test data correctly, called test_data, to be used for a Random Forest Classifier. This dictionary will be used to create a DataFrame called test_df.

This is a dictionary called ingred_value_test where the keys in the dictionary is all the ingredients in the dataset as keys. The values are 0 or 1 depending on if the particular id mentioned has the ingredient. 

NOTE: as talked about above, the test dataset does not have any cuisines. 

In [ ]:
ingred_value_test = {ingredient: [] for ingredient in unique_ingredients_test}


In [ ]:
## for test data: 
ids = []
ingredient_lst = unique_ingredients_test
ingredients_for_id =[] 
for x in test_data: 
    id_=x["id"]
    ingredients_in_id = x['ingredients']
    ids.append(id_)
    for ingred in ingred_value_test: 
            if ingred in ingredients_in_id:
                ingred_value_test[ingred].append(1)
            else: 
                 ingred_value_test[ingred].append(0)

                

In [ ]:
# create a key in the ingred_value_test dictionary called id
# The value is now a list of ids in the dataset 
# the list called ids was created above 

ingred_value_test["id"] = ids

#### e) Created a DataFrame called test_df with the ingred_value_test dictionary that was created above. 

In [ ]:
test_df = pd.DataFrame(ingred_value_test)

In [ ]:
test_df.head()

## 3) Split train data DataFrame, called train_df, into variables X and y. X is all the variables, in this case ingredients, that will be used to predict the type of cuisine. The variable y is the categorical variable cuisine. 
- X is the features, in this case ingredients, that will be used to try and figure out the classification. 
- The values of variable y is what we are trying to predict in this challenge: the type of cuisine given ingredients. 

In [ ]:
y = train_df["cuisine"]
X = train_df.drop("cuisine", axis = 1) #all variables 

## 4) Assign all the ingredients from the test data DataFrame, called test_df, to the variable X_test. X_test is all the variables, in this case ingredients, that will be used to predict the type of cuisine. I am going to drop "id". 
- X_test is the features, in this case ingredients, that will be used to try and figure out the classification.  

In [ ]:
X_test = test_df.drop("id", axis = 1)

## 5) Create a Random Forest Classifier using the package called sklearn on the train data. 

In [ ]:
rf_model = skens.RandomForestClassifier(n_estimators=10,oob_score=True, criterion='entropy')


#### a) Using the Random Forest Classifier created, called rf_model, fit the data using the train data: X and y. 

In [ ]:
rf_model.fit(X,y)

#### b)Perform cross validation using the Random Forest Classifier object, called rf_model, and the train data, called X and y, to get an accuracy score for this model. 

In [ ]:
scores = cross_val_score(rf_model, X,y, cv=5) #do this 5 times and withhold info each tim 
#score is the average score for each of the cross validaiton 

In [ ]:
#Get an accuracy score using the mean and standard deviation multiplied by 2
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

## 6) Get predicted labels for the test data 

#### a) Filter X_train and X_test to only have columns that are represented in both DataFrames

In [ ]:
# X[~X.isin(X_test.columns())]
columns_in_train = set(X.columns)
columns_in_test = set(X_test.columns)
columns_in_both = list(columns_in_train & columns_in_test)
new_df_with_columns_in_both_train = X[columns_in_both]
new_df_with_columns_in_both_test = X_test[columns_in_both]

#### b) Create a random forest classifier object, and then fit the model with new_df_with_columns_in_both_train and y

In [ ]:
rf_model_for_test = skens.RandomForestClassifier(n_estimators=10,
                                                oob_score=True, 
                                                criterion='entropy')

In [ ]:
rf_model_for_test.fit(new_df_with_columns_in_both_train,y)

#### c) Get predicted labels for new_df_with_columns_in_both_test 

In [ ]:
predicted_y_for_test = rf_model_for_test.predict(new_df_with_columns_in_both_test)

#### d) Create a copy of the test_df and assign it to the variable test_df_predicted. Add a column to test_df_predicted with the predicted labels, called predicted_y_for_test. 

In [ ]:
test_df_predicted = test_df.copy()

test_df_predicted["predicted_labels"] = predicted_y_for_test

In [ ]:
test_df_predicted.head()

#### e) Perform cross validation using the Random Forest Classifier object, called rf_model_for_test, and the train data, called new_df_with_columns_in_both_train and y, to get an accuracy score for this model. 

In [ ]:
scores_of_new_model = cross_val_score(rf_model_for_test, new_df_with_columns_in_both_train,y, cv=5)

In [ ]:
print("Accuracy: %0.2f (+/- %0.2f)" % (scores_of_new_model.mean(), scores_of_new_model.std() * 2))